In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.core import Document
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent

from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv, find_dotenv 
import os 
from llama_index.llms.groq import Groq 

import gradio as gr
import pandas as pd 
from pathlib import Path

import inspect 

from fpdf import FPDF
import tempfile
import unidecode

c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def gerar_pdf(texto):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    for linha in texto.split("\n"):
        pdf.multi_cell(0, 10, linha)

    # Cria o arquivo temporário com extensão .pdf
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    caminho = tmp.name
    tmp.close()

    # Salva o conteúdo do PDF nesse caminho
    pdf.output(caminho)

    return caminho


In [8]:
caminho_csv = "Notas_alunos.csv"
df = pd.read_csv(caminho_csv)
df[(df["aluno"] == "Ana Clara Reis Da Silva")][["nota_1bim", "nota_2bim"]].mean(axis=1)

0      7.90
28     7.75
56     8.90
84     6.15
112    7.05
140    7.85
168    7.45
196    7.35
224    7.00
252    6.60
280    6.05
308    7.15
dtype: float64

In [9]:
parametros_usados = {} 


def normalizar(texto):
    return unidecode.unidecode(str(texto)).strip().lower()

def consultar_notas(params: dict):
    caminho_csv = "Notas_alunos.csv"
    df = pd.read_csv(caminho_csv)

    turma = params.get("turma")
    ano_letivo = params.get("ano_letivo")
    matriculas = params.get("matriculas")
    alunos = params.get("alunos") or params.get("aluno")
    disciplina = params.get("disciplina")
    bimestre = params.get("bimestre")
    nota_maxima = params.get("nota_maxima", 7.0)
    tipo_consulta = params.get("tipo")  # "maior" ou "menor"
    media = params.get("media")

    df_filtrado = df.copy()

    # Normaliza colunas relevantes para garantir comparações robustas
    df_filtrado['aluno_normalizado'] = df_filtrado['aluno'].apply(normalizar)
    df_filtrado['disciplina_normalizada'] = df_filtrado['disciplina'].apply(normalizar)
    df_filtrado['turma_normalizada'] = df_filtrado['turma'].apply(normalizar)

    if alunos:
        if isinstance(alunos, str):
            alunos = [alunos]
        alunos_normalizados = [normalizar(a) for a in alunos]
        parametros_usados["alunos"] = alunos
        df_filtrado = df_filtrado[df_filtrado['aluno_normalizado'].apply(lambda x: any(a in x for a in alunos_normalizados))]
        
        if df_filtrado.empty:
            return f"Aluno(s) não encontrado(s): {', '.join(alunos)}"

    if disciplina:
        disciplina_norm = normalizar(disciplina)
        parametros_usados["disciplina"] = disciplina
        df_filtrado = df_filtrado[df_filtrado['disciplina_normalizada'] == disciplina_norm]
        
        if df_filtrado.empty:
            return f"A disciplina '{disciplina}' não foi encontrada nos registros filtrados."

    if turma:
        turma_norm = normalizar(turma)
        parametros_usados["turma"] = turma
        df_filtrado = df_filtrado[df_filtrado['turma_normalizada'] == turma_norm]

        if df_filtrado.empty:
            return f"A turma '{turma}' não foi encontrada na base de dados."

    if ano_letivo:
        parametros_usados["ano_letivo"] = ano_letivo
        df_filtrado = df_filtrado[df_filtrado['ano_letivo'] == int(ano_letivo)]

        if df_filtrado.empty:
            return f"Ano letivo '{ano_letivo}' não encontrado com os demais filtros."

    if matriculas:
        parametros_usados["matriculas"] = matriculas
        df_filtrado = df_filtrado[df_filtrado['matricula'].isin(matriculas)]

        if df_filtrado.empty:
            return f"Nenhuma matrícula encontrada: {', '.join(map(str, matriculas))}"


    if df_filtrado.empty:
        return f"Nenhum aluno encontrado com os filtros aplicados: {parametros_usados}"

    # Consulta por maior/menor nota entre os bimestres
    if tipo_consulta in ["maior", "menor"]:
        notas_colunas = [col for col in df_filtrado.columns if col.startswith("nota_")]
        resultados = []

        for _, row in df_filtrado.iterrows():
            notas_validas = [(col, row[col]) for col in notas_colunas if pd.notna(row[col])]
            if not notas_validas:
                continue

            col_alvo, valor_alvo = max(notas_validas, key=lambda x: x[1]) if tipo_consulta == "maior" else min(notas_validas, key=lambda x: x[1])
            bimestre_desc = col_alvo.replace("nota_", "").replace("bim", "")

            resultados.append(
                f"{row['aluno']} teve a {tipo_consulta} nota {valor_alvo} em {row['disciplina']} no {bimestre_desc}"
            )

            parametros_usados["nota"] = valor_alvo
            parametros_usados["bimestre"] = bimestre_desc

        return "\n".join(resultados) if resultados else "Nenhuma nota encontrada."

    if media is True:
        notas_colunas = [col for col in df_filtrado.columns if col.startswith("nota_")]

        if notas_colunas:
            # Calcula a média por linha (por aluno)
            df_filtrado["media"] = df_filtrado[notas_colunas].mean(axis=1)
            parametros_usados["media"] = "calculada"
        else:
            return "Não há colunas de nota no DataFrame."

    # Consulta padrão por bimestre
    if bimestre:
        coluna_nota = f'nota_{bimestre}bim'
        if coluna_nota not in df_filtrado.columns:
            return f"Bimestre inválido. Coluna esperada: '{coluna_nota}' não encontrada."
        
        parametros_usados["bimestre"] = bimestre

        # Só filtra por nota se nota_maxima for usada explicitamente no prompt
        if "nota_maxima" in params:
            df_filtrado = df_filtrado[df_filtrado[coluna_nota] < float(nota_maxima)]

        # Garante que existam dados para a coluna de nota após o filtro (opcional)
        if df_filtrado[coluna_nota].isnull().all():
            return f"Nenhuma nota registrada no {bimestre}º bimestre para os filtros aplicados."


    respostas = []
    for _, row in df_filtrado.iterrows():
        nota = row.get(f'nota_{bimestre}bim') if bimestre else "N/A"
        if "media" in df_filtrado.columns:
            respostas.append(
                f"{row['aluno']} (Matrícula: {row['matricula']}) - Média: {row['media']:.2f} - Disciplina: {row['disciplina']} - Turma: {row['turma']}"
            )
        else:
            respostas.append(
                f"{row['aluno']} (Matrícula: {row['matricula']}) - Nota: {nota} no {bimestre}º bimestre - Disciplina: {row['disciplina']} - Turma: {row['turma']}"
            )


    return "\n".join(respostas)

In [10]:
df = pd.read_csv("aulas_todas_materias.csv")
documents = []

for _, row in df.iterrows():
    content = f"Aula: {row['data']}\n\n{row['texto']}" 
    metadata = {
        'data': row['data'],
        'ano': row['ano'],
        'bimestre': f"{row['bimestre']}° bimestre",
        'materia': row['materia'].lower()
    }
    doc = Document(text=content, metadata=metadata)
    documents.append(doc)

documents


[Document(id_='2ec64497-dae0-4958-828e-ae2a7c2786b5', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Aula: 2 ABR\n\n2 ABR\r\nOBJETIVOS DE APRENDIZAGEM \r\nDO DC-GOEM - (GO-\r\nEMMAT501A) Compreender o \r\nconceito de função polinomial do 1º\r\ngrau, identificando a relação entre \r\nduas variáveis apresentadas em\r\ntextos de origem socioeconômicas \r\ne/ou de natureza técnico ou\r\ncientífica, entre outros para resolver \r\nsituações problemas do cotidiano.\r\nOBJETOS DE CONHECIMENTO\r\nDO DC-GOEM - Funções\r\npolinomiais do 1º\r\ngrau (função afim,\r\nfunção linear,\r\nfunção constante,\r\nfunção identidade)\r\nAplicação de Atividades via \r\nrecursos digitais\r\nRealização de Atividades via recursos \r\ndigitais', path=N

In [11]:
print(documents[0].get_metadata_str())

data: 2 ABR
ano: 2024
bimestre: 2° bimestre
materia: matematica


### Documentos em aprtes menores

In [12]:
node_parser = SentenceSplitter(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 295/295 [00:00<00:00, 1186.74it/s]


### Gerando Embeddings

In [13]:
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)
        self.name = model_name
    
    def __call__(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

In [14]:
current_file = Path().resolve()
model_folder = current_file / "all-MiniLM-L6-v2"
str(model_folder)

'C:\\Users\\user\\Documents\\projetos\\python\\tcc\\projeto\\all-MiniLM-L6-v2'

In [15]:
chroma_client = chromadb.Client()

In [16]:
try:
    chroma_collection = chroma_client.get_or_create_collection("doccuments_llm")
except Exception as e:
    print(f"Erro ao carregar ou criar embeddings {e}")

### Salvando Embeddings no DB

In [17]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [18]:
embed_model = HuggingFaceEmbedding(model_name=str(model_folder))

In [19]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

### Verificar conteúdos semelhantes entre bimestres

In [20]:
def get_text_for_bimestre(bimestre, materia, query_engine):
    prompt = f"Conteúdo do {bimestre}º bimestre de {materia}"
    response = query_engine.query(prompt)
    if hasattr(response, "response"):
        return response.response
    elif isinstance(response, str):
        return response
    else:
        return None
    
def get_embedding(text, embed_model):
    return embed_model([text])[0]


def check_revision_needed(bimestre_atual, materia, query_engine, embed_model, threshold=0.7):
    if bimestre_atual <= 1:
        return False
    
    texto_atual = get_text_for_bimestre(bimestre_atual, materia, query_engine)
    texto_anterior = get_text_for_bimestre(bimestre_atual - 1, materia, query_engine)

    if not texto_atual or not texto_anterior:
        return False
    
    emb_atual = get_embedding(texto_atual, embed_model)  # usa a função com embed_documents
    emb_ant = get_embedding(texto_anterior, embed_model)
    
    sim = cosine_similarity([emb_atual], [emb_ant])[0][0]
    print(f"Similaridade entre bimestre {bimestre_atual} e {bimestre_atual - 1} para {materia}: {sim:.4f}")
    
    return sim >= threshold


### Recuperação de informações

In [21]:
load_dotenv(find_dotenv())

True

In [22]:
GROQ_API = os.environ.get("GROQ_API")

In [23]:
llm = Groq(
    model="llama3-70b-8192",
    api_key=GROQ_API
)

In [24]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=6)

#### Testando o modelo

In [25]:
from rapidfuzz import fuzz

In [26]:
def gerar_plano_estudo(parametros_usados):
    nota = float(parametros_usados.get('nota', 0))
    bimestre = parametros_usados.get('bimestre')
    disciplina = parametros_usados.get('disciplina')
    aluno = parametros_usados.get('alunos', 'aluno')

    #if isinstance(aluno, list):
    #    aluno = aluno[0]

    if not bimestre or not disciplina:
        return "Informações insuficientes para gerar o plano de estudo (bimestre ou matéria ausente)."

    #if nota >= 8:
    #    return f"O aluno {aluno} teve nota {nota} no {bimestre}º bimestre. Nenhum plano de estudo é necessário."

    # Filtra os documentos do bimestre e matéria corretos
    docs_filtrados = [
        doc for doc in documents
        if doc.metadata['bimestre'] == f"{bimestre}° bimestre"
        and fuzz.ratio(doc.metadata.get('materia', '').lower(), disciplina.lower()) >= 80
    ]

    if not docs_filtrados:
        return "Nenhum conteúdo encontrado para esse bimestre/matéria."

    # Cria novo index e query_engine só com os docs filtrados
    nodes_filtrados = node_parser.get_nodes_from_documents(docs_filtrados)
    index_filtrado = VectorStoreIndex(nodes_filtrados, embed_model=embed_model)
    query_engine_filtrado = index_filtrado.as_query_engine(llm=llm, similarity_top_k=6)

    prompt = f"""
    {parametros_usados}
    Você é um tutor inteligente. Gere um plano de estudo **EXCLUSIVAMENTE** com base no conteúdo de {disciplina.upper()} do **{bimestre}º bimestre**.

    Ignore completamente conteúdos de outros bimestres, mesmo que relacionados.
    Use **apenas** o que foi ensinado no {bimestre}º bimestre da matéria {disciplina.upper()}.

    Se o aluno tem uma nota acima de 8 não precisa criar o plano de aula, diga que o aluno já tem uma boa noção do conteúdo.

    Com base nisso, crie:
    - Conteúdos principais a revisar
    - Dicas de como estudar
    - Sugestões de atividades práticas

    Tudo em português e focado no nível do aluno com nota {nota} e ignore a data do texto, focando apenas no bimestre.
    """

    resp = query_engine_filtrado.query(prompt)
    return resp.response


In [27]:
nota_tool = FunctionTool.from_defaults(
    fn=consultar_notas,
    name="ConsultarNotas",
    description="""
    Consulta notas de alunos a partir de um dicionário de parâmetros.

    Parâmetros possíveis:
    - aluno (str): nome do aluno
    - alunos (list[str]): lista de nomes
    - disciplina (str): nome da disciplina
    - bimestre (str ou int): bimestre (1 a 4)
    - tipo (str): 'maior' ou 'menor' para encontrar a maior ou menor nota do aluno
    - turma (str): nome da turma
    - ano_letivo (int): ano letivo
    - nota_maxima (float): filtro de nota máxima para alertas
    - nota (float): filtro de nota 
    - media (bool): média das notas 
    Exemplos de uso:
    - "Qual a menor nota do aluno João?" → {'aluno': 'João', 'tipo': 'menor'}
    - "Nota de Maria no 2º bimestre em matemática" → {'aluno': 'Maria', 'bimestre': '2', 'disciplina': 'matemática'}
    - "Media das notas dos 4 bimestre do aluno Pedro" → {'aluno': 'Pedro', 'disciplina': 'português'}
    """
)


In [ ]:
'''retriever = index.as_retriever(similarity_top_k=6)
retriever_tool = QueryEngineTool.from_defaults(
    #query_engine=retriever,
    query_engine=index.as_query_engine(llm=llm, similarity_top_k=6),
    name="Planos de aula",
    description="Ferramenta que busca planos de aula, conteúdos pedagógicos e atividades relacionados à disciplina, bimestre, turma ou aluno. Use esta ferramenta para qualquer consulta sobre plano de ensino, conteúdo, planejamento, atividades ou reforço escolar.",
)'''

In [28]:

chat_engine = ReActAgent.from_tools(
    tools=[nota_tool],
    llm=llm,
    verbose=True
)
#index.as_query_engine(llm=llm,)


c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\llama_index\core\agent\react\base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\deprecated\classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


In [29]:
def responder(mensagem, chat_history):
    if "plano de aula" in mensagem:
        resposta = gerar_plano_estudo(parametros_usados)
        caminho = gerar_pdf(resposta)
        resposta = "Gerando pdf...\nsalvo em: " + caminho 
    else:
        resposta = chat_engine.chat(mensagem).response

    if chat_history is None:
        chat_history = []

    chat_history.append(("Você", mensagem))
    chat_history.append(("Assistente", resposta))
    
    return chat_history, chat_history

In [30]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(placeholder="Digite sua pergunta ou comando...")
    estado = gr.State([])


    entrada.submit(responder, inputs=[entrada, estado], outputs=[chatbot, estado])
    entrada.submit(lambda: "", None, entrada)

demo.launch(share=True)

C:\Users\user\AppData\Local\Temp\ipykernel_21056\1295171537.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/07/28 13:27:12 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


> Running step dd0d6624-5b74-4d83-8db2-29eea5315056. Step input: me passa as notas da Ana Clara Reis Da Silva do 1 bimestre em matematica e portugues
Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: ConsultarNotas
Action Input: {'params': AttributedDict([('aluno', 'Ana Clara Reis Da Silva'), ('bimestre', '1'), ('disciplina', ['matematica', 'portugues'])])}
Observation: A disciplina '['matematica', 'portugues']' não foi encontrada nos registros filtrados.
> Running step a293e884-97e2-4f5d-b1fb-4bc918a8ea82. Step input: None
Thought: The tool responded with an error, I need to adjust the input to get the correct result.
Action: ConsultarNotas
Action Input: {'params': AttributedDict([('aluno', 'Ana Clara Reis Da Silva'), ('bimestre', '1'), ('disciplina', 'matematica')])}
Observation: Ana Clara Reis Da Silva (Matrícula: 2412752317-6) - Nota: 7.9 no 1º bimestre - Disciplina: MATEMÁTICA - Turma: 1A
> Running step ea8587d3-

Traceback (most recent call last):
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\blocks.py", line 2235, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\blocks.py", line 1746, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Document

> Running step a972acbc-ae13-4875-a5b1-ff24f2b328f6. Step input: me passa as notas da Ana Clara Reis Da Silva em matematica
Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: ConsultarNotas
Action Input: {'params': AttributedDict([('aluno', 'Ana Clara Reis Da Silva'), ('disciplina', 'matematica')])}
Observation: Ana Clara Reis Da Silva (Matrícula: 2412752317-6) - Nota: N/A no Noneº bimestre - Disciplina: MATEMÁTICA - Turma: 1A
> Running step 04dd1904-d572-4ce4-9b03-a3e54ae8c0a4. Step input: None
Thought: The tool did not return the expected result, it seems that Ana Clara Reis Da Silva does not have a grade in mathematics in the current bimester.
Action: ConsultarNotas
Action Input: {'params': AttributedDict([('aluno', 'Ana Clara Reis Da Silva'), ('disciplina', 'matematica'), ('bimestre', '1')])}
Observation: Ana Clara Reis Da Silva (Matrícula: 2412752317-6) - Nota: 7.9 no 1º bimestre - Disciplina: MATEMÁTICA - Turma: 

Traceback (most recent call last):
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\blocks.py", line 2235, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\blocks.py", line 1746, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Document

In [102]:
parametros_usados


{'alunos': ['ANA CLARA REIS DA SILVA'],
 'media': 'calculada',
 'disciplina': 'matemática',
 'nota': 6.0,
 'bimestre': '1',
 'turma': '1A'}